# Signal-Based Hedge Engine

A **modular signal aggregator** that takes your market signals and outputs direction + magnitude.

**Key concepts:**
1. You provide **signals** (any market data you care about)
2. Each signal is normalized to **[-1, +1]** (bearish to bullish)
3. You assign **weights** to each signal
4. Engine combines signals → **direction + magnitude** recommendation
5. **Position size scaling**: larger positions get more aggressive recommendations

Formula: `magnitude = signal × confidence × (1 + k × √(position_size / baseline))`

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from hedge_engine.signals import (
    HedgeEngine,
    Signal,
    Direction,
    normalize_funding_rate,
    normalize_orderbook_imbalance,
    normalize_volatility_regime,
    normalize_zscore,
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

---

## 1. Basic Usage

Add signals, compute recommendation.

In [ ]:
# Create engine
engine = HedgeEngine()

# Add signals (value: -1 to +1, weight: 0 to 1)
engine.add_signal("funding_rate", value=-0.4, weight=0.4)   # Bearish: high funding = crowded longs
engine.add_signal("orderbook", value=0.2, weight=0.3)       # Slightly bullish: more bids
engine.add_signal("momentum", value=-0.3, weight=0.3)       # Bearish: negative recent returns

# Compute recommendation for a LONG position
result = engine.compute(position_delta=500_000)  # $500k long

print("=" * 60)
print("HEDGE RECOMMENDATION")
print("=" * 60)
print(f"")
print(f"Combined Signal: {result.combined_signal:.3f}")
print(f"Confidence: {result.confidence:.1%}")
print(f"")
print(f"Direction: {result.direction.value}")
print(f"Magnitude: {result.magnitude:.1%}")
print(f"")
print(f"Signal Breakdown:")
for name, contrib in result.signal_breakdown.items():
    print(f"  {name}: {contrib:+.3f}")
print(f"")
print(f"Reasoning: {result.reasoning}")

---

## 2. Position Size Scaling

The same signal gives **different magnitude recommendations** based on position size.

Formula: `magnitude *= (1 + k × √(position_size / baseline))`

- Small positions ($10k): less aggressive
- Large positions ($1M): more aggressive de-risking

In [ ]:
# Same signals, different position sizes
engine = HedgeEngine(
    baseline_position_usd=100_000,  # $100k baseline
    size_scaling_k=0.5,             # Scaling constant
)
engine.add_signal("signal_a", value=-0.5, weight=0.6)  # Bearish signal
engine.add_signal("signal_b", value=-0.3, weight=0.4)  # Bearish signal

position_sizes = [10_000, 50_000, 100_000, 500_000, 1_000_000, 5_000_000]
results = []

for size in position_sizes:
    res = engine.compute(position_delta=size)
    results.append({
        'Position ($k)': size / 1000,
        'Magnitude': res.magnitude,
        'Direction': res.direction.value,
    })

df = pd.DataFrame(results)
print(df.to_string(index=False))

In [ ]:
# Visualize position size scaling
sizes = np.logspace(4, 7, 50)  # $10k to $10M
magnitudes = []

for size in sizes:
    res = engine.compute(position_delta=size)
    magnitudes.append(res.magnitude)

fig, ax = plt.subplots(figsize=(10, 5))
ax.semilogx(sizes / 1000, [m * 100 for m in magnitudes], 'b-', linewidth=2)
ax.fill_between(sizes / 1000, 0, [m * 100 for m in magnitudes], alpha=0.3)

ax.axvline(x=100, color='gray', linestyle='--', label='Baseline ($100k)')
ax.set_xlabel('Position Size ($k, log scale)')
ax.set_ylabel('Recommended Hedge %')
ax.set_title('Hedge Magnitude vs Position Size\n(Same bearish signal, different position sizes)')
ax.legend()
ax.set_ylim(0, 110)

plt.tight_layout()
plt.show()

print("\nKey insight: Larger positions get more aggressive hedge recommendations")
print("This is derived from: loss_threshold = k × √(position_size)")

---

## 3. Real Data: Funding Rate Signal

**Funding rate** is a real market signal:
- Positive funding: longs pay shorts → market is crowded long → bearish signal
- Negative funding: shorts pay longs → market is crowded short → bullish signal

In [ ]:
# Simulate realistic funding rate data (Binance-style, 8h funding)
# In reality, you'd fetch this from an exchange API

# Example: BTC funding rates over time (8-hour rates, annualized)
funding_rates = [
    0.02,   # Normal
    0.03,   # Slightly elevated
    0.05,   # High - crowded longs
    0.08,   # Very high - extreme long crowding
    0.12,   # Extreme - FOMO territory
    0.05,   # Coming down
    -0.01,  # Flipped negative - shorts paying
    -0.03,  # Crowded shorts
    0.00,   # Neutral
    0.02,   # Back to normal
]

# Convert to signals
signals = [normalize_funding_rate(fr, neutral=0.02, extreme=0.10) for fr in funding_rates]

df_funding = pd.DataFrame({
    'Period': range(1, len(funding_rates) + 1),
    'Funding Rate (%)': [fr * 100 for fr in funding_rates],
    'Signal': signals,
})

print("Funding Rate → Signal Conversion")
print("(Positive signal = bullish, Negative = bearish)")
print()
print(df_funding.to_string(index=False))

In [ ]:
# Visualize
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

ax1 = axes[0]
ax1.bar(df_funding['Period'], df_funding['Funding Rate (%)'], 
        color=['green' if x < 2 else 'orange' if x < 5 else 'red' for x in df_funding['Funding Rate (%)']])
ax1.axhline(y=2, color='gray', linestyle='--', label='Neutral (2%)')
ax1.set_ylabel('Funding Rate (%)')
ax1.set_title('Raw Funding Rate Data')
ax1.legend()

ax2 = axes[1]
colors = ['green' if s > 0 else 'red' for s in df_funding['Signal']]
ax2.bar(df_funding['Period'], df_funding['Signal'], color=colors)
ax2.axhline(y=0, color='gray', linestyle='-')
ax2.set_ylabel('Normalized Signal')
ax2.set_xlabel('Period')
ax2.set_title('Converted Signal (-1 = bearish, +1 = bullish)')
ax2.set_ylim(-1.1, 1.1)

plt.tight_layout()
plt.show()

In [ ]:
# Use funding signal in engine
engine = HedgeEngine()

# Period 5: extreme funding (12%)
funding_signal = normalize_funding_rate(0.12, neutral=0.02, extreme=0.10)
print(f"Funding rate: 12% → Signal: {funding_signal:.2f} (very bearish)")

engine.add_signal("funding_rate", value=funding_signal, weight=0.7)

# If you're LONG during high funding...
result = engine.compute(position_delta=300_000)  # $300k long

print(f"\nYou're LONG $300k during 12% funding:")
print(f"  → {result.direction.value} by {result.magnitude:.0%}")
print(f"  → {result.reasoning}")

---

## 4. Real Data: Orderbook Imbalance

**Orderbook imbalance** measures buying vs selling pressure:
- More bids than asks → bullish (buyers waiting)
- More asks than bids → bearish (sellers waiting)

In [ ]:
# Simulate orderbook snapshots
orderbook_data = [
    {'bid_depth': 5_000_000, 'ask_depth': 5_000_000},  # Balanced
    {'bid_depth': 6_000_000, 'ask_depth': 4_000_000},  # More bids
    {'bid_depth': 7_500_000, 'ask_depth': 2_500_000},  # Strong bid wall
    {'bid_depth': 3_000_000, 'ask_depth': 7_000_000},  # More asks
    {'bid_depth': 2_000_000, 'ask_depth': 8_000_000},  # Heavy selling pressure
]

print("Orderbook Imbalance → Signal")
print()
for i, ob in enumerate(orderbook_data):
    signal = normalize_orderbook_imbalance(ob['bid_depth'], ob['ask_depth'])
    total = ob['bid_depth'] + ob['ask_depth']
    bid_pct = ob['bid_depth'] / total * 100
    print(f"  Snapshot {i+1}: Bids ${ob['bid_depth']/1e6:.1f}M ({bid_pct:.0f}%) | Asks ${ob['ask_depth']/1e6:.1f}M → Signal: {signal:+.2f}")

---

## 5. Combining Multiple Signals

The power of the engine is combining signals with different weights.

In [ ]:
# Scenario: Mixed signals
engine = HedgeEngine()

# Signal 1: Funding rate is high (bearish)
funding_signal = normalize_funding_rate(0.08)  # 8% funding
engine.add_signal(
    "funding_rate",
    value=funding_signal,
    weight=0.4,
    description="High funding = crowded longs = bearish"
)

# Signal 2: Orderbook shows bid support (bullish)
ob_signal = normalize_orderbook_imbalance(7_000_000, 4_000_000)
engine.add_signal(
    "orderbook",
    value=ob_signal,
    weight=0.3,
    description="More bids than asks = bullish"
)

# Signal 3: Low volatility (risk-on, bullish for holding)
vol_signal = normalize_volatility_regime(0.03)  # 3% vol
engine.add_signal(
    "vol_regime",
    value=vol_signal,
    weight=0.3,
    description="Low vol = risk-on = ok to hold"
)

print("Input Signals:")
for name, sig in engine.signals.items():
    print(f"  {name}: value={sig.value:+.2f}, weight={sig.weight:.1f}")

# You're long $200k
result = engine.compute(position_delta=200_000)

print(f"\nCombined Signal: {result.combined_signal:+.3f}")
print(f"Signal Breakdown:")
for name, contrib in result.signal_breakdown.items():
    print(f"  {name}: {contrib:+.3f}")
print(f"\nConfidence: {result.confidence:.1%}")
print(f"\n→ {result.direction.value}")
print(f"→ Magnitude: {result.magnitude:.0%}")
print(f"\n{result.reasoning}")

---

## 6. Signal Agreement / Confidence

When signals disagree, **confidence is lower** → smaller recommendations.

In [ ]:
# Scenario A: Signals AGREE (all bearish)
engine_agree = HedgeEngine()
engine_agree.add_signal("sig1", value=-0.6, weight=0.5)
engine_agree.add_signal("sig2", value=-0.5, weight=0.5)
result_agree = engine_agree.compute(position_delta=100_000)

# Scenario B: Signals CONFLICT
engine_conflict = HedgeEngine()
engine_conflict.add_signal("sig1", value=-0.6, weight=0.5)  # Bearish
engine_conflict.add_signal("sig2", value=+0.5, weight=0.5)  # Bullish
result_conflict = engine_conflict.compute(position_delta=100_000)

print("Scenario A: Signals AGREE (both bearish)")
print(f"  Combined: {result_agree.combined_signal:+.2f}")
print(f"  Confidence: {result_agree.confidence:.1%}")
print(f"  → {result_agree.direction.value} by {result_agree.magnitude:.0%}")
print()
print("Scenario B: Signals CONFLICT (one bearish, one bullish)")
print(f"  Combined: {result_conflict.combined_signal:+.2f}")
print(f"  Confidence: {result_conflict.confidence:.1%}")
print(f"  → {result_conflict.direction.value}")
print()
print("Note: Lower confidence → smaller action (or HOLD)")

---

## 7. Interactive: Build Your Own Signal Mix

In [ ]:
# ========== MODIFY THESE ==========

# Your position (positive = long, negative = short)
MY_POSITION_USD = 500_000  # $500k long

# Your signals (value: -1 to +1, weight: 0 to 1)
MY_SIGNALS = [
    {"name": "funding", "value": -0.3, "weight": 0.4},     # Slightly bearish
    {"name": "orderbook", "value": 0.5, "weight": 0.3},   # Bullish
    {"name": "sentiment", "value": -0.2, "weight": 0.2},  # Slightly bearish
    {"name": "momentum", "value": 0.1, "weight": 0.1},    # Neutral-ish
]

# ===================================

engine = HedgeEngine()
for sig in MY_SIGNALS:
    engine.add_signal(sig["name"], value=sig["value"], weight=sig["weight"])

result = engine.compute(position_delta=MY_POSITION_USD)

print("=" * 60)
print("YOUR SCENARIO")
print("=" * 60)
print(f"")
print(f"Position: {'LONG' if MY_POSITION_USD > 0 else 'SHORT'} ${abs(MY_POSITION_USD):,}")
print(f"")
print("Signals:")
for sig in MY_SIGNALS:
    bar = '█' * int(abs(sig['value']) * 10)
    direction = '+' if sig['value'] > 0 else '-'
    print(f"  {sig['name']:12} {direction}{bar:10} (val={sig['value']:+.1f}, wt={sig['weight']:.1f})")
print(f"")
print(f"Combined Signal: {result.combined_signal:+.3f}")
print(f"Confidence: {result.confidence:.0%}")
print(f"")
print(f"→ RECOMMENDATION: {result.direction.value}")
print(f"→ MAGNITUDE: {result.magnitude:.0%}")
print(f"")
print(f"{result.reasoning}")
print("=" * 60)

---

## 8. Sweep: How Signal Strength Affects Recommendations

Let's see how varying a single signal affects the output.

In [ ]:
# Sweep the funding signal from -1 to +1, see recommendations
signal_values = np.linspace(-1, 1, 41)
results = []

for sv in signal_values:
    engine = HedgeEngine()
    engine.add_signal("main_signal", value=sv, weight=1.0)  # Single signal
    
    # Test with LONG position
    res = engine.compute(position_delta=200_000)
    
    # Convert direction to numeric for plotting
    if res.direction in [Direction.REDUCE_LONG, Direction.REDUCE_SHORT]:
        action_mag = -res.magnitude  # Reducing = negative
    elif res.direction in [Direction.INCREASE_LONG, Direction.INCREASE_SHORT]:
        action_mag = res.magnitude  # Increasing = positive
    else:
        action_mag = 0
    
    results.append({
        'signal': sv,
        'direction': res.direction.value,
        'magnitude': res.magnitude,
        'action_mag': action_mag,
    })

df_sweep = pd.DataFrame(results)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Action vs Signal
ax1 = axes[0]
colors = ['green' if x > 0 else 'red' if x < 0 else 'gray' for x in df_sweep['action_mag']]
ax1.bar(df_sweep['signal'], df_sweep['action_mag'] * 100, color=colors, width=0.05)
ax1.axhline(y=0, color='black', linewidth=0.5)
ax1.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax1.set_xlabel('Signal Value (-1=bearish, +1=bullish)')
ax1.set_ylabel('Action (+ = add, - = reduce)')
ax1.set_title('Recommended Action vs Signal\n(LONG $200k position)')
ax1.set_ylim(-110, 110)

# Add annotations
ax1.text(-0.8, 80, 'REDUCE LONG\n(signal bearish)', ha='center', fontsize=10, color='red')
ax1.text(0.8, 80, 'INCREASE LONG\n(signal bullish)', ha='center', fontsize=10, color='green')
ax1.axvspan(-1, -0.1, alpha=0.1, color='red', label='Reduce zone')
ax1.axvspan(-0.1, 0.1, alpha=0.1, color='gray', label='Hold zone')
ax1.axvspan(0.1, 1, alpha=0.1, color='green', label='Add zone')

# Plot 2: Magnitude vs Signal
ax2 = axes[1]
ax2.plot(df_sweep['signal'], df_sweep['magnitude'] * 100, 'b-', linewidth=2)
ax2.axhline(y=0, color='black', linewidth=0.5)
ax2.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Signal Value')
ax2.set_ylabel('Magnitude (%)')
ax2.set_title('Recommendation Magnitude vs Signal')

plt.tight_layout()
plt.show()

---

## Summary

### How It Works
1. **Signals**: You provide any market signals, normalized to [-1, +1]
2. **Weights**: You decide how much to trust each signal (auto-normalized)
3. **Aggregation**: Weighted average -> combined signal
4. **Position Context**: Given your position, determines direction
5. **Size Scaling**: Larger positions -> more aggressive recommendations

### Key Formula
```
magnitude = signal_strength x confidence x (1 + k x sqrt(position_size / baseline))
```

### Signal Normalizers Available
- `normalize_funding_rate()`: High funding = bearish
- `normalize_orderbook_imbalance()`: More bids = bullish
- `normalize_volatility_regime()`: High vol = risk-off
- `normalize_zscore()`: Generic z-score conversion

### Distributional Signals (NEW)
- `quick_downside_metrics(er, vol)`: Get P(loss), CVaR, Kelly from forecast
- `quick_signal(er, vol)`: Convert distribution to [-1, +1] signal

### What YOU Provide
- Your position (long/short, size)
- Your signals (any market data)
- Your weights (relative importance)

The engine does NOT tell you what signals matter - **that's your job as the trader**.

---

## 9. Distributional Signals: Downside Risk

Instead of point estimates, use **full probability distributions** to compute:
- **P(loss)**: Probability of negative return
- **CVaR**: Expected loss in the worst 5%
- **Kelly**: Optimal position sizing

In [ ]:
from hedge_engine.distributional import quick_downside_metrics, quick_signal

# Your model predicts: expected return and volatility
scenarios = [
    {"name": "Good bet", "er": 0.03, "vol": 0.05},
    {"name": "Risky bet", "er": 0.03, "vol": 0.15},
    {"name": "Bad bet", "er": -0.01, "vol": 0.05},
]

print("Downside Risk Metrics")
print("=" * 70)
print(f"{'Scenario':<12} {'E[r]':>8} {'Vol':>8} {'P(loss)':>10} {'CVaR 5%':>10} {'Kelly':>10}")
print("-" * 70)

for s in scenarios:
    metrics = quick_downside_metrics(s["er"], s["vol"])
    print(f"{s['name']:<12} {s['er']:>7.1%} {s['vol']:>7.1%} {metrics.p_loss:>9.1%} {metrics.cvar_5:>9.2%} {metrics.kelly:>9.0%}")

print()
print("Key insight: Same expected return, different risk profiles!")

In [ ]:
# Convert distribution to signal and use in engine
print("Distribution -> Signal -> Hedge Recommendation")
print("=" * 60)

# Your model's forecast
expected_return = 0.015  # 1.5%
volatility = 0.08        # 8%

metrics = quick_downside_metrics(expected_return, volatility)
signal_value = quick_signal(expected_return, volatility, method='probability')

print(f"\nYour forecast: E[r]={expected_return:.1%}, vol={volatility:.1%}")
print(f"  P(loss): {metrics.p_loss:.1%}")
print(f"  CVaR 5%: {metrics.cvar_5:.2%}")
print(f"  Half-Kelly: {metrics.half_kelly:.0%}")
print(f"  -> Signal: {signal_value:+.2f}")

# Combine with other signals
engine = HedgeEngine()
engine.add_signal("model_forecast", value=signal_value, weight=0.5)
engine.add_signal("funding_rate", value=-0.4, weight=0.3)  # Bearish
engine.add_signal("orderbook", value=0.2, weight=0.2)      # Slightly bullish

result = engine.compute(position_delta=400_000)

print(f"\nCombined with other signals:")
print(f"  -> {result.direction.value} by {result.magnitude:.0%}")

---

## 10. Backtesting with Real Data

Validate signals on actual market data with realistic transaction costs.

In [ ]:
from hedge_engine.backtest import (
    Backtest,
    fetch_crypto_data,
    momentum_signal,
    volatility_regime_signal,
    trend_following_signal,
)

# Fetch real BTC data from CoinGecko (free, no API key)
print("Fetching real BTC price data...")
prices, timestamps = fetch_crypto_data("bitcoin", days=365)
print(f"Got {len(prices)} days: {timestamps[0].date()} to {timestamps[-1].date()}")
print(f"Price range: ${min(prices):,.0f} to ${max(prices):,.0f}")
print()

# Run backtest with multiple signals
bt = Backtest(prices, timestamps)
bt.add_signal("momentum", lambda p, i: momentum_signal(p, i, lookback=20), weight=0.4)
bt.add_signal("volatility", lambda p, i: volatility_regime_signal(p, i), weight=0.3)
bt.add_signal("trend", lambda p, i: trend_following_signal(p, i), weight=0.3)

results = bt.run()
print(results.summary())

---

## 11. Optuna Weight Calibration

Optimize signal weights for best out-of-sample performance.

In [ ]:
# pip install optuna  (if not installed)
from hedge_engine.calibrate import calibrate_weights, DEFAULT_SIGNALS

# Use same price data from above
print("Running Optuna calibration (30 trials)...")
print("Optimizing for best Sharpe ratio on training data")
print()

result = calibrate_weights(
    prices,
    timestamps,
    signals=DEFAULT_SIGNALS,
    n_trials=30,
    train_pct=0.6,  # 60% train, 40% test
    metric="sharpe",
    verbose=False,
)

print(result.summary())

print("\nKey insight: Compare Train vs Test Sharpe to detect overfitting!")
print("If Test << Train, the weights don't generalize.")